In [5]:
!pip install nltk rouge
!git clone https://github.com/salaniz/pycocoevalcap

fatal: destination path 'pycocoevalcap' already exists and is not an empty directory.


In [6]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import json
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize

# Load JSON data
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

data = load_json('/content/drive/MyDrive/MasterThesis/captions_data_combined_all.json')

# Ensure NLTK's resources are downloaded
nltk.download('punkt')
nltk.download('wordnet')

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

# Function to calculate BLEU score with multiple references
def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

# Initialize ROUGE scorer
rouge = Rouge()

# Prepare to accumulate scores for averaging
model_scores = {
    'blip': {'bleu': [], 'meteor': [], 'rouge': []},
    'gpt2': {'bleu': [], 'meteor': [], 'rouge': []},
    'pix2struct': {'bleu': [], 'meteor': [], 'rouge': []}
}

# Calculate scores for each model
for item in data.values():
    original_captions = item['original_coco_captions']
    for model_name, generated_caption in item['generated_captions'].items():

        bleu_score = calculate_corpus_bleu(original_captions, generated_caption)
        meteor = calculate_meteor(original_captions[0], generated_caption)
        rouge_scores = rouge.get_scores(generated_caption, original_captions[0])[0]
        model_scores[model_name]['bleu'].append(bleu_score)
        model_scores[model_name]['meteor'].append(meteor)
        model_scores[model_name]['rouge'].append(rouge_scores)

# Print average scores for each model
for model_name, scores in model_scores.items():
    avg_bleu = sum(scores['bleu']) / len(scores['bleu']) if scores['bleu'] else 0
    avg_meteor = sum(scores['meteor']) / len(scores['meteor']) if scores['meteor'] else 0

    # Initialize sums for ROUGE scores
    sum_rouge_1f = sum_rouge_2f = sum_rouge_lf = 0

    for rouge_score in scores['rouge']:
        sum_rouge_1f += rouge_score['rouge-1']['f']
        sum_rouge_2f += rouge_score['rouge-2']['f']
        sum_rouge_lf += rouge_score['rouge-l']['f']

    len_rouge = len(scores['rouge']) if scores['rouge'] else 1  # Avoid division by zero
    avg_rouge_1f = sum_rouge_1f / len_rouge
    avg_rouge_2f = sum_rouge_2f / len_rouge
    avg_rouge_lf = sum_rouge_lf / len_rouge

    print(f"Model: {model_name}")
    print(f"Average BLEU: {avg_bleu:.4f}")
    print(f"Average METEOR: {avg_meteor:.4f}")
    print(f"Average ROUGE-1 F: {avg_rouge_1f:.4f}")
    print(f"Average ROUGE-2 F: {avg_rouge_2f:.4f}")
    print(f"Average ROUGE-L F: {avg_rouge_lf:.4f}\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Model: blip
Average BLEU: 0.3053
Average METEOR: 0.3021
Average ROUGE-1 F: 0.4015
Average ROUGE-2 F: 0.1626
Average ROUGE-L F: 0.3692

Model: gpt2
Average BLEU: 0.3166
Average METEOR: 0.3269
Average ROUGE-1 F: 0.4042
Average ROUGE-2 F: 0.1579
Average ROUGE-L F: 0.3691

Model: pix2struct
Average BLEU: 0.0542
Average METEOR: 0.1652
Average ROUGE-1 F: 0.2268
Average ROUGE-2 F: 0.0289
Average ROUGE-L F: 0.2062



In [ ]:
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python
import re

tool = language_tool_python.LanguageTool('en-US')



def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/Scenario1_distilbart_with_all_similarities_only_2.json')
data2 = load_json('/content/drive/MyDrive/MasterThesis/Scenario2_distilbart_with_all_similarities_only_2.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]
    item2 = data2[key]

    chosen_item = item1 if item1['cosine_similarity_score'] > item2['cosine_similarity_score'] else item2

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Save to a new JSON file
with open('/content/drive/MyDrive/MasterThesis/comparison_results_with_scores_scenario1_distilbart_only2_corrected.json', 'w') as f:
    json.dump(results, f, indent=4)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Load the computed scores from the JSON file
results = load_json('/content/drive/MyDrive/MasterThesis/comparison_results_with_scores_scenario1_distilbart_only2.json')

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

Average BLEU Score: 0.24657408616091894
Average METEOR Score: 0.3242025912459397
Average ROUGE-1 F Score: 0.42430698754114216
Average ROUGE-2 F Score: 0.16511749291219777
Average ROUGE-L F Score: 0.382525370575479


In [ ]:
# Load the computed scores from the JSON file
results = load_json('/content/drive/MyDrive/MasterThesis/comparison_results_with_scores_scenario1_distilbart_only2_corrected.json')

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

Average BLEU Score: 0.28125906603559464
Average METEOR Score: 0.3096670317265307
Average ROUGE-1 F Score: 0.43948117493769295
Average ROUGE-2 F Score: 0.1712963804957992
Average ROUGE-L F Score: 0.4012980962077358


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')
import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_2.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.22826707700430982
Average METEOR Score: 0.3342978864673826
Average ROUGE-1 F Score: 0.4286275689598294
Average ROUGE-2 F Score: 0.15932962587548197
Average ROUGE-L F Score: 0.3973841762285319


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')
import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_2.json' )

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(corrected_sentence)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Average BLEU Score: 0.22274421718797743
Average METEOR Score: 0.3164986091568829
Average ROUGE-1 F Score: 0.4282903802823139
Average ROUGE-2 F Score: 0.1562247561349688
Average ROUGE-L F Score: 0.396180633194704


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_3.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.27880925859577826
Average METEOR Score: 0.31383242920217486
Average ROUGE-1 F Score: 0.42567616485692744
Average ROUGE-2 F Score: 0.16149316369211097
Average ROUGE-L F Score: 0.3889367150266809


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_3.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.27013597652551935
Average METEOR Score: 0.2958808879732082
Average ROUGE-1 F Score: 0.4279811827784412
Average ROUGE-2 F Score: 0.16272253225932354
Average ROUGE-L F Score: 0.39216820658184126


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_2.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.26425651508547876
Average METEOR Score: 0.2984825986159144
Average ROUGE-1 F Score: 0.4220652306102396
Average ROUGE-2 F Score: 0.1598498678015676
Average ROUGE-L F Score: 0.3907658869148601


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_4.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.2756003182641043
Average METEOR Score: 0.31161028731468465
Average ROUGE-1 F Score: 0.4217115909616362
Average ROUGE-2 F Score: 0.16136976295267105
Average ROUGE-L F Score: 0.38459112748380037


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python
import re

tool = language_tool_python.LanguageToolPublicAPI('en-US')



def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_4.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.268642421388139
Average METEOR Score: 0.2975805732464576
Average ROUGE-1 F Score: 0.4253431479847537
Average ROUGE-2 F Score: 0.1613088175289083
Average ROUGE-L F Score: 0.38931952729018515


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_5.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.27115359153745733
Average METEOR Score: 0.30602015275481714
Average ROUGE-1 F Score: 0.4288312371938363
Average ROUGE-2 F Score: 0.16399374331713323
Average ROUGE-L F Score: 0.3925275988929338


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1_5.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.27642257748063226
Average METEOR Score: 0.3172805251624054
Average ROUGE-1 F Score: 0.4251202863803479
Average ROUGE-2 F Score: 0.16195678833839833
Average ROUGE-L F Score: 0.38840358640550177


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_2_blip_1.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.27009759462962735
Average METEOR Score: 0.3206946507296894
Average ROUGE-1 F Score: 0.42646983723757925
Average ROUGE-2 F Score: 0.16415067995429708
Average ROUGE-L F Score: 0.3867183896423783


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Average BLEU Score: 0.26609000241166036
Average METEOR Score: 0.3144647767133634
Average ROUGE-1 F Score: 0.4225124435283919
Average ROUGE-2 F Score: 0.1601385484599433
Average ROUGE-L F Score: 0.38394173059700193


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_ann.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Average BLEU Score: 0.2800738105536158
Average METEOR Score: 0.34646685990795806
Average ROUGE-1 F Score: 0.46238936788978696
Average ROUGE-2 F Score: 0.18735978384362822
Average ROUGE-L F Score: 0.4272587292404049


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_gpt2_1_blip_2_3.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Average BLEU Score: 0.27015988211641206
Average METEOR Score: 0.295818586393029
Average ROUGE-1 F Score: 0.42801652026725595
Average ROUGE-2 F Score: 0.16238517000828653
Average ROUGE-L F Score: 0.39200899893334373


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_ann_1.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Average BLEU Score: 0.2643751105768122
Average METEOR Score: 0.32129850663936615
Average ROUGE-1 F Score: 0.42426255940585084
Average ROUGE-2 F Score: 0.16263674513699358
Average ROUGE-L F Score: 0.38436754913323523


In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import language_tool_python

tool = language_tool_python.LanguageToolPublicAPI('en-US')

import re

def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_ann_summary_modified.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches if a sentence ends with ", a"
    pattern = r',\s+a\s*$'
    # Remove the pattern if found at the end of the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence


rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)
    print(trimmed_and_corrected_sentence)
    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


A hot dog
 A table with a plate of hot dogs
 A table with a plate of hot dogs
 A table with a plate of hot dogs
 A table with a plate of hot dogs
 A person riding a motorcycle with a dog on the back of people with
A group of people riding scoots on a street a number
A dog laying in a dirty bathroom in a filthy bathroom.
A man riding a bike on a city street is seen
 A bathroom with marble walls
A dog is standing in the bathroom with a toilet a
A dog laying on the ground
A little girl sitting on the ground, a young boy girl.
A black dog laying on the floor in a kitchen dog.
A woman standing in a kitchen with a dog
A dog wearing a shirt
A dog is riding a bike with a basket a
A dog is sitting in a basket on a bike
A dog in a car dog.
 A dog
A dog sitting in the back seat of a car sitting.
Two dogs are looking out the window of a car, two
A man and woman walking a dog
A dog on a motorcycle with a helmet
 A dog wearing a helmet
A dog is laying on the floor next to a motorcycle
A black dog la

In [ ]:
###########Using weighted summarized version
import json
import nltk
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
!pip install language_tool_python
import requests
from PIL import Image
from io import BytesIO
import language_tool_python
import matplotlib.pyplot as plt
import re

tool = language_tool_python.LanguageToolPublicAPI('en-US')



def remove_trailing_phrases(sentence):
    # Remove trailing " and a", " a", "of a", "with a", or "in a" at the end of a sentence
    patterns = [
        r'\s+and\s+a\s*$',
        r'\s+a\s*$',
        r'\s+of\s+a\s*$',
        r'\s+with\s+a\s*$',
        r'\s+in\s+a\s*$'
    ]
    for pattern in patterns:
        sentence = re.sub(pattern, '', sentence)
    return sentence

def remove_repeating_parts(sentence):
    words = sentence.split()
    if len(words) > 3:
        first_three_words = set(words[:3])
        for i, word in enumerate(words[3:], start=3):
            if word in first_three_words:
                return ' '.join(words[:i])
    return sentence

def remove_and_a_following(sentence):
    # Remove everything after "and a" including "and a"
    pattern = r"\s+and\s+a\s+.*"
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def remove_pattern(sentence):
    # This regex pattern targets " is a " followed by exactly one word.
    # \s matches any whitespace, \w+ matches one or more word characters (letters, digits, or underscore),
    # and the space before \w+ ensures we're only capturing one word following " is a".
    pattern = r"\s+is\s+a\s+\w+"
    # Replace the pattern with an empty string to remove it from the sentence.
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence

def correct_spelling_and_grammar(sentence):
    matches = tool.check(sentence)
    corrected_sentence = language_tool_python.utils.correct(sentence, matches)
    return corrected_sentence

def trim_sentence_at_full_stop(sentence):
    parts = sentence.split('.')
    trimmed_sentence = parts[0].strip() + '.' if len(parts) > 1 else parts[0]
    return trimmed_sentence

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# Load the JSON data
data1 = load_json('/content/drive/MyDrive/MasterThesis/weighted_summarization_ann_summary_modified.json')

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

def calculate_corpus_bleu(references, candidate):
    references_tokenized = [[word_tokenize(ref.lower()) for ref in references]]
    candidate_tokenized = [word_tokenize(candidate.lower())]
    smoothing_function = SmoothingFunction().method1
    return corpus_bleu(references_tokenized, candidate_tokenized, smoothing_function=smoothing_function)

def calculate_meteor(reference, candidate):
    reference_tokenized = word_tokenize(reference.lower())
    candidate_tokenized = word_tokenize(candidate.lower())
    return meteor_score([reference_tokenized], candidate_tokenized)

def remove_comma_a_ending(sentence):
    # This pattern matches if a sentence ends with ", a"
    pattern = r',\s+a\s*$'
    # Remove the pattern if found at the end of the sentence
    modified_sentence = re.sub(pattern, '', sentence)
    return modified_sentence


rouge = Rouge()

results = {}
for key in data1:
    item1 = data1[key]


    chosen_item = item1

    summarized_sentence = chosen_item.get('most_similar_summarized_sentence', chosen_item.get('summarized_caption', ''))

    corrected_sentence = correct_spelling_and_grammar(summarized_sentence)
    sentence_without_pattern = remove_pattern(corrected_sentence)
    sentence_without_trailing_phrases = remove_trailing_phrases(sentence_without_pattern)
    sentence_without_and_a = remove_and_a_following(sentence_without_trailing_phrases)
    sentence_without_repeating_parts = remove_repeating_parts(sentence_without_and_a)
    trimmed_and_corrected_sentence = trim_sentence_at_full_stop(sentence_without_repeating_parts)

    trimmed_and_corrected_sentence = remove_comma_a_ending(trimmed_and_corrected_sentence)
    original_captions = chosen_item['original_coco_captions']
    image_url = chosen_item.get('url', 'No URL found')

    bleu_score = calculate_corpus_bleu(original_captions, trimmed_and_corrected_sentence)
    meteor = calculate_meteor(original_captions[0], trimmed_and_corrected_sentence)
    rouge_scores = rouge.get_scores(trimmed_and_corrected_sentence, original_captions[0])[0]

    if bleu_score>=0.6 and meteor>=0.6:
      response = requests.get(image_url)
      img = Image.open(BytesIO(response.content))
      plt.figure(figsize=(10, 10))
      plt.imshow(img)
      plt.axis('off')
      plt.show()

      print(chosen_item['generated_captions'])
      print('Summarized caption'+str(summarized_sentence))
      print(original_captions)
      print("ANN caption:"+str( trimmed_and_corrected_sentence))
      print(bleu_score)
      print(meteor)
      print(rouge_scores)


    results[key] = {
        'corrected_summarized_sentence': trimmed_and_corrected_sentence,
        'original_captions': original_captions,
        'cosine_similarity_score': chosen_item['cosine_similarity_score'],
        'bleu': bleu_score,
        'meteor': meteor,
        'rouge': rouge_scores,
        'image_url': image_url
    }

# Variables to accumulate scores
total_bleu = 0
total_meteor = 0
total_rouge_1_f = 0
total_rouge_2_f = 0
total_rouge_l_f = 0
num_items = len(results)

# Iterate through the results to accumulate scores
for key, value in results.items():
    total_bleu += value['bleu']
    total_meteor += value['meteor']
    total_rouge_1_f += value['rouge']['rouge-1']['f']
    total_rouge_2_f += value['rouge']['rouge-2']['f']
    total_rouge_l_f += value['rouge']['rouge-l']['f']

# Calculate averages
average_bleu = total_bleu / num_items
average_meteor = total_meteor / num_items
average_rouge_1_f = total_rouge_1_f / num_items
average_rouge_2_f = total_rouge_2_f / num_items
average_rouge_l_f = total_rouge_l_f / num_items

# Print averages
print(f"Average BLEU Score: {average_bleu}")
print(f"Average METEOR Score: {average_meteor}")
print(f"Average ROUGE-1 F Score: {average_rouge_1_f}")
print(f"Average ROUGE-2 F Score: {average_rouge_2_f}")
print(f"Average ROUGE-L F Score: {average_rouge_l_f}")

Output hidden; open in https://colab.research.google.com to view.